In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', None)  # Show all column content
pd.set_option('display.max_columns', None)  # Show all columns

In [3]:
df = pd.read_csv("reddit_posts_comments_Depression.csv")

In [4]:
df_post = df['Post Title']
df_body = df['Post Body']
df_comment = df['Comment']

In [5]:
df_post = pd.DataFrame(df_post)
df_post.reset_index(drop=True)
print()

In [6]:
df_body = pd.DataFrame(df_body)
df_body.reset_index(drop=True)
print()

In [7]:
df_comment = pd.DataFrame(df_comment)
df_comment.reset_index(drop=True)
print()

In [8]:
from warnings import filterwarnings
filterwarnings(action='ignore')

In [9]:
df_post['Post Title'] = df_post['Post Title'].apply(lambda m: m.strip().lower())
df_body['Post Body'] = df_body['Post Body'].apply(lambda m: m.strip().lower())
df_comment['Comment'] = df_comment['Comment'].apply(lambda m: m.strip().lower())

In [10]:
import re

# Function to clean and retain only words (strings)
def clean_text(x):
    if isinstance(x, str):  # Ensure it's a string
        # Use regex to extract words, join them with spaces, and convert to lowercase
        return " ".join(re.findall(r'\b\w+', x))
    return None  # Discard non-strings

# Apply the cleaning function to the DataFrame columns
df_post['Post Title'] = df_post['Post Title'].apply(clean_text)
df_body['Post Body'] = df_body['Post Body'].apply(clean_text)
df_comment['Comment'] = df_comment['Comment'].apply(clean_text)


In [11]:
df_post['Post Title'] = df_post['Post Title'].apply(lambda text: re.sub(r'\s+', ' ',re.sub(r'_', ' ', re.sub(r'\d+', '', text))).strip())

df_body['Post Body'] = df_body['Post Body'].apply(lambda text: re.sub(r'\s+', ' ',re.sub(r'_', ' ', re.sub(r'\d+', '', text))).strip())

df_comment['Comment'] = df_comment['Comment'].apply(lambda text: re.sub(r'\s+', ' ',re.sub(r'_', ' ', re.sub(r'\d+', '', text))).strip())

## Tokenization

In [12]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TreebankWordTokenizer

In [13]:
tokenizer = TreebankWordTokenizer()

In [14]:
import nltk
# nltk.download('punkt')

In [15]:
df_post['Post Title'] = df_post['Post Title'].apply(tokenizer.tokenize)
df_comment['Comment'] = df_comment['Comment'].apply(tokenizer.tokenize)

In [16]:
from nltk.corpus import stopwords

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\navne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop_words = stopwords.words('english')

In [19]:
def remove_stopwords(message):
    global stop_words
    result = []
    for word in str(message).split():  # Ensure the input is a string
        if word.lower().strip() not in stop_words:
            result.append(word)
    return " ".join(result)

# Apply the function to each entry in the Post Body column
df_post['Post Title'] = df_post['Post Title'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)  # Convert lists to strings
df_post['Post Title'] = df_post['Post Title'].apply(remove_stopwords)

df_comment['Comment'] = df_comment['Comment'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)  # Convert lists to strings
df_comment['Comment'] = df_comment['Comment'].apply(remove_stopwords)

In [20]:
# pip install spacy

In [21]:
# !python -m spacy download en_core_web_sm

In [22]:
# pip install numpy==1.26.4

In [23]:
#df_post['Post Title'] = df_post['Post Title'].apply(tokenizer.tokenize)
#df_comment['Comment'] = df_comment['Comment'].apply(tokenizer.tokenize)

In [24]:
df_comment.head(3)

,Comment
0,understood apologise forget future break
1,removed
2,biggest problem private talks may may reach point advice one needs help leave chat would even make worse public chat good chance others may jump got lost


In [25]:
df_comment = df_comment[df_comment["Comment"].apply(lambda x: x != ["removed"])]

In [26]:
df_comment.reset_index(drop=True)
print()

In [27]:
df_post = df_post.drop_duplicates()

In [28]:
df_post = df_post.reset_index(drop=True)
print()

In [29]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\navne\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
import spacy

# Initialize the spaCy model and lemmatizer
nlp = spacy.load("en_core_web_sm")
lemmatizer = WordNetLemmatizer()

# Function to process each row
def process_row(text):
    if isinstance(text, str):  # If the row is a string
        words = text.split()  # Split by spaces to get words
    elif isinstance(text, list):  # If the row is a list
        words = text
    else:
        return text  # Return as is for other types

    # Process the text with spaCy to get POS tagging
    doc = nlp(' '.join(words))  # Convert the list of words back to text for spaCy processing

    seen = set()
    lemmatized_words = []
    
    # Loop through the words and apply lemmatization based on POS tags
    for token in doc:
        if token.pos_ in ['ADJ', 'VERB', 'NOUN']:  # Check if the word is an adjective or verb
            lemma = token.lemma_.lower()  # Lemmatize and convert to lowercase
            if lemma not in seen:  # Add only unique lemmas
                seen.add(lemma)
                lemmatized_words.append(lemma)
    
    return lemmatized_words if isinstance(text, list) else ' '.join(lemmatized_words)

# Apply the function to the 'Comment' and 'Post Title' columns
df_comment['Comment'] = df_comment['Comment'].apply(process_row)
df_post['Post Title'] = df_post['Post Title'].apply(process_row)


In [31]:
import pandas as pd
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize and remove similar words
def remove_similar_words(words):
    # Set to store words and remove duplicates
    lemmatized_words = set()
    
    # Apply lemmatization and store base form in set to avoid duplicates
    for word in words:
        lemma = lemmatizer.lemmatize(word.lower())  # Lemmatize and lowercase the word
        lemmatized_words.add(lemma)
    
    # Return the unique lemmatized words
    return list(lemmatized_words)

# Apply the function to the 'Comment' column
df_comment['Comment'] = df_comment['Comment'].apply(remove_similar_words)


In [32]:
df_post.head(3)

,Post Title
0,break least understand rule helper invite private contact first resort make new wiki explain
1,regular check post information rule wikis
2,psychiatrist make sign suicide contract
